In [ ]:
9#google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [6]:
import pandas as pd
df=pd.read_csv('/content/master_job_scam_data.csv')
df.head()

,title,description,requirements,company_profile,location,salary_range,employment_type,industry,benefits,fraudulent
0,OUD: SummaView zoekt een Business Controller 9,SummaView is opzoek naar een (interim) busines...,Je maakt als SummaView Specialists deel uit va...,SummaView is een jong bedrijf uit Amersfoort d...,"NL, UT, Amersfoort",NaN,NaN,NaN,Je aanmelden als kandidaat voor vaste functies...,0
1,Office Manager,"WebLinc is looking for a motivated, energetic ...",Requirements:Intermediate to advanced experien...,WebLinc is the e-commerce platform and service...,"US, PA, Philadelphia",40000-45000,Full-time,Internet,Health &amp; WellnessMedical planPrescription ...,0
2,Legal executive,She hundred technology. Earn $5000/week! Immed...,"Basic knowledge in entire, no degree required....",Miller-Hardin - Established 2011.,Lake Matthew,$66562-$77696,Contract,Automotive,Sign-on bonus,1
3,SQL Developer,SQL Developer FLSA: ExemptReports to: IT Man...,Required SkillsBachelor’s Degree in Computer ...,"Although AssistRx was formed in only 2009, we ...","US, FL, Orlando",70000-85000,Full-time,Computer Software,Benefits Highlight Health and Dental Insurance...,0
4,Applications developer,Soon someone particularly car. Earn $5000/week...,"Basic knowledge in seven, no degree required. ...",Zimmerman-Erickson - Established 1973.,Bentonville,$53984-$97099,Internship,IT,Remote work opportunities,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27566 entries, 0 to 27565
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            27566 non-null  object
 1   description      27565 non-null  object
 2   requirements     24920 non-null  object
 3   company_profile  24292 non-null  object
 4   location         27225 non-null  object
 5   salary_range     12818 non-null  object
 6   employment_type  24141 non-null  object
 7   industry         22734 non-null  object
 8   benefits         20466 non-null  object
 9   fraudulent       27566 non-null  int64 
dtypes: int64(1), object(9)
memory usage: 2.1+ MB


In [8]:
df.columns

Index(['title', 'description', 'requirements', 'company_profile', 'location',
       'salary_range', 'employment_type', 'industry', 'benefits',
       'fraudulent'],
      dtype='object')

In [9]:
# Fill missing text fields with 'Not Provided'
text_cols = ['title', 'location', 'company_profile',
             'description', 'requirements', 'benefits', 'employment_type',
              'industry']

for col in text_cols:
    df[col] = df[col].fillna("Not Provided")

# Fill missing numeric or boolean fields (if any)
df['salary_range'] = df['salary_range'].fillna("Not Provided")


In [10]:
df['text_data'] = (
    df['title'] + ' ' +
    df['company_profile'] + ' ' +
    df['description'] + ' ' +
    df['requirements'] + ' ' +
    df['benefits']
)


In [11]:
def clean_text(text):
    text = text.lower()                             # Lowercase
    text = re.sub(r'http\S+', '', text)             # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)         # Remove punctuation/numbers
    text = re.sub(r'\s+', ' ', text).strip()        # Remove extra spaces
    return text

df['clean_text'] = df['text_data'].apply(clean_text)


In [17]:
#same amount of fradulant and non fradulant
import pandas as pd

# assume df is your dataframe
# and 'fraudulent' is the target column

fraud_df = df[df['fraudulent'] == 1]
real_df = df[df['fraudulent'] == 0]

# make both have equal size = size of the smaller class
real_df_balanced = real_df.sample(n=len(fraud_df), random_state=42)
df_balanced = pd.concat([fraud_df, real_df_balanced])

# shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print(df_balanced['fraudulent'].value_counts())



fraudulent
0    10854
1    10854
Name: count, dtype: int64


In [18]:
# Target variable (fraudulent)
df_balanced['fraudulent'] = df_balanced['fraudulent'].astype(int)


In [19]:
#count of fraud
df_balanced['fraudulent'].value_counts()

,count
fraudulent,
0,10854
1,10854


In [20]:
X = df_balanced['clean_text']
y = df_balanced['fraudulent']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [21]:
tfidf = TfidfVectorizer(stop_words='english', max_features=10000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [22]:
print("Train:", X_train_tfidf.shape)
print("Test:", X_test_tfidf.shape)

Train: (17366, 10000)
Test: (4342, 10000)


In [23]:
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train_tfidf, y_train)


RandomForestClassifier(n_estimators=200, random_state=42)

In [24]:
y_pred = model.predict(X_test_tfidf)

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      2138
           1       1.00      0.96      0.98      2204

    accuracy                           0.98      4342
   macro avg       0.98      0.98      0.98      4342
weighted avg       0.98      0.98      0.98      4342

Confusion Matrix:
 [[2137    1]
 [  84 2120]]


In [25]:
import joblib

joblib.dump(model, 'fake_job_detector.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']